In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
with open('C:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/data/06_models/lightgbm_model.pkl', 'rb') as file:
    model = pickle.load(file)

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
data_path = "C:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/data/03_primary/df_agg.csv"

In [4]:
data = pd.read_csv(data_path)

C:\Users\Z478SG\AppData\Local\Temp\ipykernel_7316\560202589.py:1: DtypeWarning: Columns (756,757,761,762,766,767,771,772,776,777,781,782,786,787,791,792) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [5]:
columns_to_remove = []

for col in data.columns:
    if data[col].dtype=='object':
        print(col)
        columns_to_remove.append(col)

CC_NAME_CONTRACT_STATUS_Active_MIN
CC_NAME_CONTRACT_STATUS_Active_MAX
CC_NAME_CONTRACT_STATUS_Approved_MIN
CC_NAME_CONTRACT_STATUS_Approved_MAX
CC_NAME_CONTRACT_STATUS_Completed_MIN
CC_NAME_CONTRACT_STATUS_Completed_MAX
CC_NAME_CONTRACT_STATUS_Demand_MIN
CC_NAME_CONTRACT_STATUS_Demand_MAX
CC_NAME_CONTRACT_STATUS_Refused_MIN
CC_NAME_CONTRACT_STATUS_Refused_MAX
CC_NAME_CONTRACT_STATUS_Sent proposal_MIN
CC_NAME_CONTRACT_STATUS_Sent proposal_MAX
CC_NAME_CONTRACT_STATUS_Signed_MIN
CC_NAME_CONTRACT_STATUS_Signed_MAX
CC_NAME_CONTRACT_STATUS_nan_MIN
CC_NAME_CONTRACT_STATUS_nan_MAX


In [6]:
data.drop(columns=columns_to_remove, inplace=True)

In [7]:
data.shape

(356251, 781)

In [8]:
for col in data.columns:
    if data[col].dtype=='object':
        print(col)

In [9]:
data.dtypes

SK_ID_CURR                              int64
TARGET                                float64
CODE_GENDER                             int64
FLAG_OWN_CAR                            int64
FLAG_OWN_REALTY                         int64
                                       ...   
CC_NAME_CONTRACT_STATUS_Signed_VAR    float64
CC_NAME_CONTRACT_STATUS_nan_MEAN      float64
CC_NAME_CONTRACT_STATUS_nan_SUM       float64
CC_NAME_CONTRACT_STATUS_nan_VAR       float64
CC_COUNT                              float64
Length: 781, dtype: object

In [10]:
data = data.astype('float16')

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [11]:
data.dtypes

SK_ID_CURR                            float16
TARGET                                float16
CODE_GENDER                           float16
FLAG_OWN_CAR                          float16
FLAG_OWN_REALTY                       float16
                                       ...   
CC_NAME_CONTRACT_STATUS_Signed_VAR    float16
CC_NAME_CONTRACT_STATUS_nan_MEAN      float16
CC_NAME_CONTRACT_STATUS_nan_SUM       float16
CC_NAME_CONTRACT_STATUS_nan_VAR       float16
CC_COUNT                              float16
Length: 781, dtype: object

In [12]:
train_data = data[data['TARGET'].notnull()]
train_data.shape

(307507, 781)

In [13]:
# voir si y'a aussi cette erreur sur l'ancien notebook

# Faire tourner un autre modèle que LGBX

In [14]:
# Avoir train_data avec la même quantité de 0 ou de 1. Et regarder si proba tjrs pareil ou pas

# Sélectionner les lignes avec TARGET = 1
# df_1 = train_data[train_data['TARGET'] == 1]

# # Sélectionner un nombre égal de lignes avec TARGET = 0
# df_0 = train_data[train_data['TARGET'] == 0].sample(n=len(df_1), random_state=42)  # random_state pour la reproductibilité

# # Concaténer les deux DataFrames
# train_data = pd.concat([df_0, df_1])

# train_data = df_balanced
# train_data.shape

In [15]:
train_data['TARGET'].describe()

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)


count    307507.000000
mean          0.000000
std           0.272461
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: TARGET, dtype: float64

In [16]:
# train_data = train_data[:10000]
# train_data.shape

In [17]:
X = train_data.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y = train_data['TARGET']

In [39]:
X.shape

(307507, 779)

In [18]:
model.predict_proba(X)

array([[0.81542823, 0.18457177],
       [0.93124454, 0.06875546],
       [0.92946442, 0.07053558],
       ...,
       [0.91975014, 0.08024986],
       [0.93879217, 0.06120783],
       [0.91847791, 0.08152209]])

In [36]:
# Étape 1: Prédire les probabilités
proba_predictions = model.predict_proba(X[y==1])[:, 1]  # Prendre la probabilité de la classe 1


In [37]:
proba_predictions.shape

(282682,)

In [38]:
print(f"Min: {proba_predictions.min()}")
print(f"Mean: {proba_predictions.mean()}")
print(f"Max: {proba_predictions.max()}")

Min: 0.0566292821336211
Mean: 0.07883655308122671
Max: 0.23194254930965344


In [21]:
# Étape 2: Appliquer un seuil pour convertir les probabilités en prédictions binaires
def apply_threshold(proba_predictions, threshold):
    return (proba_predictions >= threshold).astype(int)

In [22]:
# Étape 3: Évaluer la précision du modèle
def evaluate_model(y_true, proba_predictions, threshold):
    binary_predictions = apply_threshold(proba_predictions, threshold)
    accuracy = accuracy_score(y_true, binary_predictions)
    return accuracy

In [23]:
# Exemple d'utilisation
threshold = 0.8  # Vous pouvez ajuster ce seuil manuellement
accuracy = evaluate_model(y, proba_predictions, threshold)
print(f"Précision du modèle avec un seuil de {threshold}: {accuracy}")

ValueError: Found input variables with inconsistent numbers of samples: [307507, 24825]

In [31]:
# Pour tester différents seuils
thresholds = np.arange(0.05, 0.24, 0.01)
for threshold in thresholds:
    accuracy = evaluate_model(y, proba_predictions, threshold)
    print(f"Précision du modèle avec un seuil de {threshold}: {accuracy}")

Précision du modèle avec un seuil de 0.05: 0.08072986956394489
Précision du modèle avec un seuil de 0.060000000000000005: 0.21680156874477655
Précision du modèle avec un seuil de 0.07: 0.514391542306354
Précision du modèle avec un seuil de 0.08000000000000002: 0.6693246007407961
Précision du modèle avec un seuil de 0.09000000000000001: 0.7578624226440374
Précision du modèle avec un seuil de 0.1: 0.820095152305476
Précision du modèle avec un seuil de 0.11000000000000001: 0.856074821060984
Précision du modèle avec un seuil de 0.12000000000000001: 0.8802401246150494
Précision du modèle avec un seuil de 0.13: 0.8954397786066659
Précision du modèle avec un seuil de 0.14: 0.904694852474903
Précision du modèle avec un seuil de 0.15000000000000002: 0.9103304965415422
Précision du modèle avec un seuil de 0.16000000000000003: 0.9143824368225764
Précision du modèle avec un seuil de 0.17000000000000004: 0.9170392869105419
Précision du modèle avec un seuil de 0.18000000000000005: 0.9184734006055146